### Análise do Índice Geral de Reclamações (IGR)
### Agência Nacional de Saúde Suplementar (ANS)

Análise exploratória do Índice de Reclamações (IGR) divulgado pela ANS, que mede a satisfação dos beneficiários de planos de saúde. O estudo traz comparações entre porte e tipo de cobertura das operadoras, além de rankings e insights sobre a relação entre número de beneficiários, reclamações e qualidade percebida.

In [ ]:
#Importando todas bibliotecas necessárias para análise e visualização dos dados

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
import plotly.express as px
import numpy as np
from io import StringIO
import plotly.graph_objects as go
from plotly.subplots import make_subplots

#abaixo estão os comandos para instalar as bibliotecas necessárias, caso não estejam instaladas no ambiente
#!pip install pandas
#!pip install matplotlib
#!pip install seaborn
#!pip install scikit-learn
#!pip install plotly
#!pip install --upgrade nbformat
#pip install tabulate

#Lendo o arquivo CSV com os dados do IGR
df_igr = pd.read_csv('arquivos/igr.csv', delimiter=';', on_bad_lines='warn', low_memory=False)
df_igr.head()

### Objetivo
Entender a distribuição do IGR no mercado.

Gráfico de Histograma → distribuição dos valores de IGR.

Boxplot → outliers e mediana do IGR por operadora.

In [ ]:
# Limpar a coluna IGR, substituindo a vírgula por ponto para converter para tipo float
df_igr['IGR'] = df_igr['IGR'].astype(str).str.replace(',', '.', regex=False).astype(float)

# Gerar o Histograma da distribuição do IGR
plt.figure(figsize=(10, 6))
plt.hist(df_igr['IGR'], bins=10, edgecolor='k', alpha=0.7)
plt.title('Distribuição do IGR', fontsize=16)
plt.xlabel('Valor do IGR', fontsize=12)
plt.ylabel('Frequência', fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.savefig('histograma_igr.png')
plt.show()
# Gerar o Boxplot do IGR por Porte da Operadora
plt.figure(figsize=(12, 8))
df_igr.boxplot(column='IGR', by='PORTE_OPERADORA')
plt.title('Boxplot do IGR por Porte da Operadora', fontsize=16)
plt.suptitle('')  # Suprimir o título automático do `by`
plt.xlabel('Porte da Operadora', fontsize=12)
plt.ylabel('Valor do IGR', fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.savefig('boxplot_igr.png')
plt.show()

print("gráficos 'boxplot_igr.png' foram gerados com sucesso.")

### Objetivo 
Comparar desempenho entre pequenas, médias e grandes.

Boxplot (IGR por porte) → mostra diferenças de dispersão.

Barplot (média do IGR por porte) → visão comparativa direta.

In [ ]:
# Limpar a coluna IGR, substituindo a vírgula por ponto para converter para tipo float
df_igr['IGR'] = df_igr['IGR'].astype(str).str.replace(',', '.', regex=False).astype(float)

# Gerar o Boxplot (IGR por Porte da Operadora)
plt.figure(figsize=(12, 8))
sns.boxplot(x='PORTE_OPERADORA', y='IGR', data=df_igr)
plt.title('Distribuição de IGR por Porte da Operadora', fontsize=16)
plt.xlabel('Porte da Operadora', fontsize=12)
plt.ylabel('IGR', fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.savefig('boxplot_igr_por_porte.png')
plt.show()
print("'boxplot_igr_por_porte.png' gerado com sucesso.")

print("Porte por operadora")
# Calcular a média do IGR por Porte da Operadora
df_avg_igr = df_igr.groupby('PORTE_OPERADORA')['IGR'].mean().reset_index()

# Gerar o Barplot (média do IGR por Porte da Operadora)
plt.figure(figsize=(10, 6))
sns.barplot(x='PORTE_OPERADORA', y='IGR', data=df_avg_igr)
plt.title('Média do IGR por Porte da Operadora', fontsize=16)
plt.xlabel('Porte da Operadora', fontsize=12)
plt.ylabel('Média do IGR', fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.savefig('barplot_avg_igr.png')
plt.show()

print("'barplot_avg_igr.png' gerado com sucesso.")

### Objetivo 
Ver se planos de saúde e odontológicos têm padrões diferentes.

Barplot (IGR médio por tipo de cobertura).

Gráfico de dispersão (Beneficiários × IGR, colorido por tipo de cobertura) → mostra concentração de casos.

In [ ]:
# Limpar a coluna IGR, substituindo a vírgula por ponto para converter para tipo float
df_igr['IGR'] = df_igr['IGR'].astype(str).str.replace(',', '.', regex=False).astype(float)


# Removendo outliers do IGR (método IQR)
Q1 = df_igr['IGR'].quantile(0.25)
Q3 = df_igr['IGR'].quantile(0.75)
IQR = Q3 - Q1
limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR

# Criando uma cópia do DataFrame sem os outliers
df_igr_clean = df_igr[(df_igr['IGR'] >= limite_inferior) & (df_igr['IGR'] <= limite_superior)].copy()

# Barplot: IGR médio por tipo de cobertura
plt.figure(figsize=(10, 6))
sns.barplot(
    x='COBERTURA',
    y='IGR',
    data=df_igr_clean.groupby('COBERTURA')['IGR'].mean().reset_index()
)
plt.title('Média do IGR por Tipo de Cobertura', fontsize=16)
plt.xlabel('Tipo de Cobertura', fontsize=12)
plt.ylabel('Média do IGR', fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.savefig('barplot_avg_igr_cobertura.png')
plt.show()
print("Gráfico 'barplot_avg_igr_cobertura.png' gerado com sucesso.")

# Normalizando os tamanhos dos pontos com base no IGR
scaler = MinMaxScaler(feature_range=(50, 500))
df_igr_clean['tamanho_ponto'] = scaler.fit_transform(df_igr_clean[['IGR']])

# Scatterplot: IGR vs Beneficiários
plt.figure(figsize=(12, 8))
sns.scatterplot(
    x='QTD_BENEFICIARIOS',
    y='IGR',
    hue='COBERTURA',
    size='tamanho_ponto',
    sizes=(50, 500),
    alpha=0.6,
    palette='Set2',
    data=df_igr_clean
)

plt.xscale("log")
plt.title('Dispersão: IGR vs. Beneficiários por Cobertura', fontsize=16)
plt.xlabel('Quantidade de Beneficiários', fontsize=12)
plt.ylabel('IGR', fontsize=12)
plt.grid(axis='both', linestyle='--', alpha=0.6)
plt.legend(title='Cobertura', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.savefig('scatterplot_beneficiarios_igr.png', bbox_inches='tight')
plt.show()
print("Gráfico 'scatterplot_beneficiarios_igr.png' gerado com sucesso.")


### Objetivo 
Entender se o tamanho da operadora impacta no índice.

Scatterplot (nº de beneficiários × nº de reclamações, tamanho do ponto = IGR) → facilita ver desproporções.

Gráfico de linha de tendência → relação entre base de clientes e índice.

In [ ]:
# Limpar a coluna IGR, substituindo a vírgula por ponto para converter para tipo float
df_igr['IGR'] = df_igr['IGR'].astype(str).str.replace(',', '.', regex=False).astype(float)

# Normalizando os tamanhos dos pontos
scaler = MinMaxScaler(feature_range=(50, 1000))  # ajusta a escala dos pontos
sizes = scaler.fit_transform(df_igr[['IGR']])[:, 0]

# Gráfico de dispersão com escala log em X e Y
plt.figure(figsize=(12, 8))
plt.scatter(
    x=df_igr['QTD_BENEFICIARIOS'],
    y=df_igr['QTD_RECLAMACOES'],
    s=sizes,
    alpha=0.6
)
plt.xscale("log")
plt.yscale("log")
plt.title('Dispersão: Reclamações vs. Beneficiários (Tamanho do ponto = IGR)', fontsize=16)
plt.xlabel('Número de Beneficiários (escala log)', fontsize=12)
plt.ylabel('Número de Reclamações (escala log)', fontsize=12)
plt.grid(which="both", linestyle='--', alpha=0.7)

# Linha de referência: 1 reclamação para cada 1000 beneficiários
x_vals = np.array([df_igr['QTD_BENEFICIARIOS'].min(), df_igr['QTD_BENEFICIARIOS'].max()])
y_vals = x_vals / 1000  # proporção 1:1000
plt.plot(x_vals, y_vals, color='red', linestyle='--', label='1 reclamação / 1000 beneficiários')

# legendas com apenas valores de referência para IGR
legend_sizes = [df_igr['IGR'].min(), df_igr['IGR'].median(), df_igr['IGR'].max()]
for size_val in legend_sizes:
    scaled_size = scaler.transform([[size_val]])[0, 0]
    plt.scatter([], [], s=scaled_size, label=f'IGR = {size_val:.2f}', alpha=0.6, color='gray')

plt.legend(scatterpoints=1, frameon=False, labelspacing=1, title='Legenda', loc='upper left')
plt.savefig('scatter_igr.png')
plt.show()
print("linhas de tendencias")

# Gráfico de linhas de tendência (regplot) com log nos eixos
plt.figure(figsize=(10, 6))
sns.regplot(
    x='QTD_BENEFICIARIOS',
    y='IGR',
    data=df_igr,
    scatter_kws={'s': 30, 'alpha': 0.5},
    line_kws={'color': 'blue'}
)
plt.xscale("log")
plt.yscale("log")
plt.title('Relação entre Número de Beneficiários e IGR', fontsize=16)
plt.xlabel('Número de Beneficiários', fontsize=12)
plt.ylabel('IGR (escala log)', fontsize=12)
plt.grid(which="both", linestyle='--', alpha=0.7)
plt.savefig('regplot_beneficiarios_igr.png')
plt.show()

print("\nOs gráficos 'scatter_igr.png' e 'regplot_beneficiarios_igr.png' foram gerados com sucesso.")


In [ ]:
# Limpar a coluna IGR, substituindo a vírgula por ponto para converter para tipo float
df_igr['IGR'] = df_igr['IGR'].astype(str).str.replace(',', '.', regex=False).astype(float)

# colunas para tamanhos dos pontos normalizados
sizes = (df_igr['IGR'] - df_igr['IGR'].min()) / (df_igr['IGR'].max() - df_igr['IGR'].min()) * 50 + 20 # entre 20 e 70
df_igr['size'] = sizes

# Categorizando IGR: baixo, médio, alto
q1 = df_igr['IGR'].quantile(0.33)
q2 = df_igr['IGR'].quantile(0.66)

def categorize_igr(val):
    if val <= q1:
        return 'Baixo'
    elif val <= q2:
        return 'Médio'
    else:
        return 'Alto'

df_igr['IGR_categoria'] = df_igr['IGR'].apply(categorize_igr)

# gráfico interativo com cores por categoria de IGR
fig = px.scatter(
    df_igr,
    x='QTD_BENEFICIARIOS',
    y='QTD_RECLAMACOES',
    size='size',
    color='IGR_categoria',
    hover_name='RAZAO_SOCIAL',
    hover_data={'IGR': True, 'QTD_BENEFICIARIOS': True, 'QTD_RECLAMACOES': True, 'RAZAO_SOCIAL': True},
    labels={
        'QTD_BENEFICIARIOS': 'Número de Beneficiários',
        'QTD_RECLAMACOES': 'Número de Reclamações',
        'IGR': 'IGR',
        'IGR_categoria': 'Categoria IGR'
    },
    title='Dispersão Interativa: Reclamações vs. Beneficiários (Tamanho do ponto = IGR)',
    log_x=True,
    log_y=True,
    color_discrete_map={'Baixo':'green', 'Médio':'orange', 'Alto':'red'}
)

# linha de referência: 1 reclamação para cada 1000 beneficiários
x_vals = np.array([df_igr['QTD_BENEFICIARIOS'].min(), df_igr['QTD_BENEFICIARIOS'].max()])
y_vals = x_vals / 1000
fig.add_traces(px.line(x=x_vals, y=y_vals).data)
fig.update_traces(selector=dict(mode='lines'), line=dict(color='blue', dash='dash'), name='1 reclamação / 1000 beneficiários')

# Layout 
fig.update_layout(
    legend_title_text='Legenda',
    template='plotly_white',
    width=1000,
    height=700
)

fig.show()

### Objetivo 
Descobrir se operadoras melhoram ou pioram ao longo do tempo.

Lineplot (IGR ao longo das competências por operadora) → para grandes players do mercado.

Heatmap (operadoras x tempo) → cores mostrando intensidade do IGR.

In [ ]:
# Função para gerar siglas únicas

def gerar_siglas_unicas(nomes, max_len=12):
    siglas = {}
    usados = set()

    for nome in nomes:
        palavras = nome.split()
        # Começa pegando as iniciais
        sigla = ''.join([p[0] for p in palavras]).upper()

        # Se for muito curta, usa até 12 chars do nome
        if len(sigla) < 3:
            sigla = (nome[:max_len]).upper()

        # Evitar duplicados
        base = sigla
        i = 1
        while sigla in usados:
            sigla = (base[:max_len-2] + str(i)).upper()
            i += 1

        usados.add(sigla)
        siglas[nome] = sigla

    return siglas

# Garantir que COMPETENCIA está em formato de data

#df_igr['COMPETENCIA'] = pd.to_datetime(df_igr['COMPETENCIA'].astype(str), format='%Y%m')
#df_igr['COMPETENCIA'] = pd.to_datetime(df_igr['COMPETENCIA'].astype(str), format='mixed')
df_igr['COMPETENCIA'] = pd.to_datetime(df_igr['COMPETENCIA'].astype(str), format='%Y%m', errors='coerce')

# Filtrar período (01/2024 até 08/2025)
df_periodo = df_igr[
    (df_igr['COMPETENCIA'] >= "2024-01-01") &
    (df_igr['COMPETENCIA'] <= "2025-08-01")
]

# Selecionar 30 operadoras (critério: mais registros no período)
top_30_operadoras = (
    df_periodo['RAZAO_SOCIAL']
    .value_counts()
    .head(30)
    .index
)

# Filtrar apenas essas operadoras
df_30 = df_periodo[df_periodo['RAZAO_SOCIAL'].isin(top_30_operadoras)]

# Remover Outliers do IGR (método IQR)
Q1 = df_30['IGR'].quantile(0.25)
Q3 = df_30['IGR'].quantile(0.75)
IQR = Q3 - Q1
limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR
df_30 = df_30[(df_30['IGR'] >= limite_inferior) & (df_30['IGR'] <= limite_superior)]


# Criar siglas únicas para as operadoras
siglas_dict = gerar_siglas_unicas(df_30['RAZAO_SOCIAL'].unique())
df_30['OPERADORA_ABREV'] = df_30['RAZAO_SOCIAL'].map(siglas_dict)

# 1. SCATTER INTERATIVO
fig_scatter = px.scatter(
    df_30,
    x="COMPETENCIA",
    y="IGR",
    color="OPERADORA_ABREV",
    trendline="lowess",
    title="Evolução Temporal do IGR (01/2024 a 08/2025) - Top 30 Operadoras"
)
fig_scatter.update_traces(mode="lines+markers")
fig_scatter.update_layout(
    xaxis_title="Competência",
    yaxis_title="IGR (quanto menor, melhor)",
    legend_title="Siglas Operadoras",
    hovermode="x unified"
)
fig_scatter.show()


# 2. HEATMAP INTERATIVO
pivot_igr = df_30.pivot_table(index='OPERADORA_ABREV', columns='COMPETENCIA', values='IGR')
pivot_igr.columns = pivot_igr.columns.strftime("%Y-%m")

fig_heatmap = px.imshow(
    pivot_igr,
    labels=dict(x="Competência", y="Siglas Operadoras", color="IGR"),
    aspect="auto",
    title="Heatmap do IGR - Top 30 Operadoras (01/2024 a 08/2025)",
    color_continuous_scale="RdYlGn_r"
)
fig_heatmap.update_xaxes(side="top")
fig_heatmap.show()

# 3. TABELA DE REFERÊNCIA (Sigla ↔ Nome Completo)
df_legenda = pd.DataFrame(list(siglas_dict.items()), columns=["Operadora", "Sigla"]).sort_values("Sigla")

fig_table = go.Figure(data=[go.Table(
    header=dict(values=["Siglas", "Operadoras"], fill_color="lightgrey", align="left"),
    cells=dict(values=[df_legenda["Sigla"], df_legenda["Operadora"]], align="left")
)])
fig_table.update_layout(title="Tabela de Referência - Siglas das Operadoras 30 ")
fig_table.show()


### Objetivo 
Identificar melhores e piores operadoras.

Ranking top 10 dos piores para os melhores.

Tabela resumo com: Operadora | Beneficiários | Reclamações | IGR.

In [ ]:
# Calcular métricas por operadora
df_resumo = (
    df_30.groupby(["RAZAO_SOCIAL", "OPERADORA_ABREV"])
    .agg({
        "IGR": "mean",                      # média do IGR no período
        "QTD_BENEFICIARIOS": "sum",         # total de beneficiários
        "QTD_RECLAMACOES": "sum"            # total de reclamações
    })
    .reset_index()
)

# Selecionar Top 10 melhores e piores
top10_melhores = df_resumo.nsmallest(10, "IGR").assign(Ranking="Melhores")
top10_piores = df_resumo.nlargest(10, "IGR").assign(Ranking="Piores")

# Concatenar em um só dataframe
df_rank = pd.concat([top10_melhores, top10_piores])

# Criar layout de subplots (1 linha, 2 colunas)
fig = make_subplots(
    rows=1, cols=2,
    column_widths=[0.6, 0.4],
    subplot_titles=("Ranking Top 10 Melhores e Piores", "Resumo das Operadoras")
)

# Barplot Ranking
bar = px.bar(
    df_rank.sort_values("IGR"),
    x="IGR", y="OPERADORA_ABREV",
    color="Ranking",
    orientation="h",
    text="IGR",
    color_discrete_map={"Melhores": "green", "Piores": "red"}
)

for trace in bar.data:
    fig.add_trace(trace, row=1, col=1)

# Tabela Resumo
df_legenda = df_resumo.sort_values("IGR")

tabela = go.Table(
    header=dict(values=["Operadora", "Sigla", "Beneficiários", "Reclamações", "IGR Médio"],
                fill_color="lightgrey", align="left"),
    cells=dict(values=[
        df_legenda["RAZAO_SOCIAL"],
        df_legenda["OPERADORA_ABREV"],
        df_legenda["QTD_BENEFICIARIOS"],
        df_legenda["QTD_RECLAMACOES"],
        df_legenda["IGR"].round(3)
    ], align="left")
)

# Criar layout de subplots (1 linha, 2 colunas)

fig = make_subplots(
    rows=1, cols=2,
    column_widths=[0.9, 0.9],
    subplot_titles=("Ranking Top 10 do Pior para o Melhor", "Resumo das Operadoras"),
    specs=[[{"type": "xy"}, {"type": "domain"}]] 
)

# Adiciona traços de gráfico de barras
for trace in bar.data:
    fig.add_trace(trace, row=1, col=1)

# Adiciona rastreamento da tabela
fig.add_trace(tabela, row=1, col=2)

fig.update_layout(
    title="Benchmark Negativo e Positivo - Operadoras (01/2024 a 08/2025)",
    height=800
)

fig.show()
fig.write_image("benchmark_operadoras.png")
